In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")
sns.set
import time

In [2]:
Start_time = time.time()

#### Defining Maximum number of Levels and No. of Simulations

In [4]:
no_of_level = 5
no_of_iteration = 10000

#### Defining Rate Parameter(Lamda) Value for each Level in each iteration

In [6]:
lamda_new = 1/500
lamda_rem_1 = 2*lamda_new
lamda_rem_2 = 4*lamda_new
lamda_rem_3 = 8*lamda_new
lamda_rem_4 = 16*lamda_new

In [7]:
#### Based on Failure data of the cooling sysetm in DieselEngine (Dhananjay R Dolas) MTTF is taken between [400,3000)
# lamda_Table = pd.DataFrame([])
# for i in range(no_of_iteration):
#     a = np.round(1/500,3)
#     lamda_Row = np.array([a])
#     for j in range(no_of_level-1):
#         lamda_Row = pd.Series(np.append(lamda_Row,2*a))
#     lamda_Table[i] = lamda_Row
# lamda_Table = lamda_Table.T
# lamda_Table.head(3)

In [8]:
lamda_Table = pd.DataFrame([np.full(no_of_iteration,lamda_new) for i in range(no_of_level)]).T
lamda_Table.head(2)

,0,1,2,3,4
0,0.002,0.002,0.002,0.002,0.002
1,0.002,0.002,0.002,0.002,0.002


#### Defining Proportion of Mixture for each Level in each iteration

In [10]:
proportion_Table = pd.DataFrame([])
alpha = np.ones(no_of_level)
for i in range(no_of_iteration):
    proportion_Row = pd.Series(np.array(np.round(np.random.dirichlet(alpha),1)))
    proportion_Table[i] = proportion_Row
proportion_Table = proportion_Table.T
proportion_Table.head(3)

,0,1,2,3,4
0,0.3,0.4,0.2,0.1,0.0
1,0.2,0.0,0.5,0.1,0.2
2,0.2,0.2,0.1,0.5,0.1


#### Genearating Random Numbers for Reliability of each Level

In [12]:
Random_Table_for_Reliability = pd.DataFrame([])
for i in range(no_of_iteration):
    Random_Row = np.array([])
    for j in range(no_of_level):
        Random_Row = pd.Series(np.append(Random_Row,np.round(np.random.rand(),4)))
    Random_Table_for_Reliability[i] = Random_Row
Random_Table_for_Reliability = Random_Table_for_Reliability.T
Random_Table_for_Reliability.head(3)

,0,1,2,3,4
0,0.6634,0.9330,0.6953,0.5502,0.6060
1,0.3367,0.4784,0.1061,0.4103,0.7792
2,0.3026,0.6246,0.0518,0.5595,0.5304


In [13]:
Random_Table_for_Reliability.head(3)

,0,1,2,3,4
0,0.6634,0.9330,0.6953,0.5502,0.6060
1,0.3367,0.4784,0.1061,0.4103,0.7792
2,0.3026,0.6246,0.0518,0.5595,0.5304


#### Calculating Failure Time for each Levels

In [15]:
Failure_Time_Table = np.round((-1 / lamda_Table)*np.log( Random_Table_for_Reliability),1)
Failure_Time_Table.head(3)

,0,1,2,3,4
0,205.2,34.7,181.7,298.7,250.4
1,544.3,368.7,1121.7,445.4,124.7
2,597.7,235.3,1480.2,290.4,317.1


#### Adding the Lamda,Proportion Columns to the Failure_Time_Table

In [17]:
Failure_Time_of_Mixture = pd.DataFrame([])
for i in range(no_of_level):
    Failure_Time_of_Mixture["Lamda of Level "+str(i+1)] = lamda_Table.iloc[:,i]
    Failure_Time_of_Mixture["Failure Time of Level "+str(i+1)] = Failure_Time_Table.iloc[:,i]
    Failure_Time_of_Mixture["Proportion of Level "+str(i+1)] = proportion_Table.iloc[:,i]

Sum_Series = np.array([])
for i in range(no_of_iteration):
    Sum_of_Proprtion = 0
    for j in range(no_of_level):
        Sum_of_Proprtion = Failure_Time_of_Mixture.loc[i,"Proportion of Level "+str(j+1)] + Sum_of_Proprtion 
    Sum_Series = np.append(Sum_Series,Sum_of_Proprtion) 
    
Failure_Time_of_Mixture["Sum of Prportion"] = pd.Series(Sum_Series)
Failure_Time_of_Mixture.head(3)

,Lamda of Level 1,Failure Time of Level 1,Proportion of Level 1,Lamda of Level 2,Failure Time of Level 2,Proportion of Level 2,Lamda of Level 3,Failure Time of Level 3,Proportion of Level 3,Lamda of Level 4,Failure Time of Level 4,Proportion of Level 4,Lamda of Level 5,Failure Time of Level 5,Proportion of Level 5,Sum of Prportion
0,0.002,205.2,0.3,0.002,34.7,0.4,0.002,181.7,0.2,0.002,298.7,0.1,0.002,250.4,0.0,1.0
1,0.002,544.3,0.2,0.002,368.7,0.0,0.002,1121.7,0.5,0.002,445.4,0.1,0.002,124.7,0.2,1.0
2,0.002,597.7,0.2,0.002,235.3,0.2,0.002,1480.2,0.1,0.002,290.4,0.5,0.002,317.1,0.1,1.1


#### Calculating Mixture Failure Time

In [19]:
Mixture_Failure_Time_Series = np.array([])
for i in range(no_of_iteration):
    Mixture_Failure_Time_Value = np.round((Failure_Time_Table*proportion_Table).iloc[i,:].sum(),1)
    Mixture_Failure_Time_Series= pd.Series(np.append(Mixture_Failure_Time_Series,Mixture_Failure_Time_Value))
Failure_Time_of_Mixture["Mixture Failure Time"] = Mixture_Failure_Time_Series
Failure_Time_of_Mixture["Lamda"] = lamda_new
(Failure_Time_of_Mixture).head(3)

,Lamda of Level 1,Failure Time of Level 1,Proportion of Level 1,Lamda of Level 2,Failure Time of Level 2,Proportion of Level 2,Lamda of Level 3,Failure Time of Level 3,Proportion of Level 3,Lamda of Level 4,Failure Time of Level 4,Proportion of Level 4,Lamda of Level 5,Failure Time of Level 5,Proportion of Level 5,Sum of Prportion,Mixture Failure Time,Lamda
0,0.002,205.2,0.3,0.002,34.7,0.4,0.002,181.7,0.2,0.002,298.7,0.1,0.002,250.4,0.0,1.0,141.6,0.002
1,0.002,544.3,0.2,0.002,368.7,0.0,0.002,1121.7,0.5,0.002,445.4,0.1,0.002,124.7,0.2,1.0,739.2,0.002
2,0.002,597.7,0.2,0.002,235.3,0.2,0.002,1480.2,0.1,0.002,290.4,0.5,0.002,317.1,0.1,1.1,491.5,0.002


#### Exporting the Data

In [21]:
name_of_csv_file = str(no_of_level) + '_Levels Component having lamda:'+str(lamda_new)+'for new one (3).csv'
Failure_Time_of_Mixture.to_csv(name_of_csv_file )

##### Total Run time 

In [23]:
End_time = time.time()
print("Time_Taken",np.round(End_time-Start_time,3))

Time_Taken 18.185
